In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

import os
import shutil
import time
from tqdm import tqdm

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

2022-11-01 06:52:45.995323: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-01 06:52:45.995365: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Load trained model
model = tf.keras.models.load_model('/home/jupyter/gurih_mas/digit_recognizer/models/DR_MobileNetV2_v3.2.h5')

# Show the model architecture
model.summary()

2022-11-01 06:52:47.854510: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-01 06:52:47.854556: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-01 06:52:47.854581: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (addin-gurihmas-dev): /proc/driver/nvidia/version does not exist
2022-11-01 06:52:47.854820: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                12810 

In [3]:
def convert_range(prediction_score, OldMin, OldMax):
    NewMax = 1
    NewMin = 0
    
    OldRange = (OldMax - OldMin)  
    NewRange = (NewMax - NewMin)  
    NewValue = (((prediction_score - OldMin) * NewRange) / OldRange) + NewMin
    return NewValue

def get_prediction_s(image_dir, model_name):
    test_img = tf.keras.utils.load_img(image_dir,
                                   grayscale = False,
                                   color_mode = 'rgb',
                                   target_size = (160,160,3),
                                   interpolation = 'nearest')
    img_array = tf.keras.utils.img_to_array(test_img)
    img_array = tf.expand_dims(img_array, 0)
    predictions = model_name.predict(img_array)
    score = predictions[0]
    
    class_names = [i for i in range(10)]
    pred_label = class_names[np.argmax(score)]
    
    conf_score = np.max(score)
    model_score = np.max(score)
    
    return pred_label, conf_score, model_score,score

In [5]:
data_images = pd.read_csv('/home/jupyter/gurih_mas/digit_recognizer/labels/221003_dataset/221003_true_label.csv',header=0, names=['ktp_pict','true_label'])
data_images

,ktp_pict,true_label
0,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_1.jpg,3.0
1,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_3.jpg,4.0
2,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_6.jpg,7.0
3,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_9.jpg,2.0
4,1f3b44fe-d75d-49cb-81e7-ba9abe1fa803_3.jpg,3.0
...,...,...
4009,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_5.jpg,8.0
4010,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_9.jpg,5.0
4011,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_11.jpg,1.0
4012,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_12.jpg,0.0


In [6]:
data_images = data_images.drop('true_label', axis=1)
data_images

,ktp_pict
0,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_1.jpg
1,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_3.jpg
2,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_6.jpg
3,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_9.jpg
4,1f3b44fe-d75d-49cb-81e7-ba9abe1fa803_3.jpg
...,...
4009,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_5.jpg
4010,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_9.jpg
4011,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_11.jpg
4012,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_12.jpg


In [7]:
pred_label = []
scoring = []

for i in tqdm(range(0, data_images.shape[0])):
    image_folder = '221003_dataset'
    image_download_directory = os.path.join("/home/jupyter/gurih_mas/digit_recognizer/dataset",image_folder)
    image_filename = data_images.ktp_pict[i]
    image_full_path_dir = '{}/{}'.format(image_download_directory, image_filename)
    
    try:
        pred_label_s, conf_score_s, model_score_s,score = get_prediction_s(image_full_path_dir, model)
        pred_label.append(pred_label_s)
        scoring.append(conf_score_s)
    
    except:
        pass

100%|██████████| 4014/4014 [04:14<00:00, 15.75it/s]


In [8]:
data_images['pred_label'] = pred_label
data_images['score'] = scoring
data_images

,ktp_pict,pred_label,score
0,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_1.jpg,3,0.989121
1,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_3.jpg,4,0.988733
2,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_6.jpg,7,1.000000
3,0d2498b5-1d8b-4255-a5d0-2ca7c8a6fe0f_9.jpg,2,0.703459
4,1f3b44fe-d75d-49cb-81e7-ba9abe1fa803_3.jpg,3,0.990398
...,...,...,...
4009,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_5.jpg,8,0.974919
4010,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_9.jpg,5,0.951002
4011,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_11.jpg,1,0.993096
4012,febbec3d-b729-4a7d-9cfd-0ec8b9ad290d_12.jpg,0,0.999984


In [9]:
data_images[['ktp_pict','pred_label']].to_csv('/home/jupyter/gurih_mas/digit_recognizer/results/DR_221003_prediction_result_v3.2.csv',index=False)